In [ ]:
import pandas as pd
import dbconnect  # dbconnect.py 모듈 활용
import os


print("[Master Data] 호선 매핑 테이블 생성")

# 1. DB 연결 (seoul_urban_lab)
# dbconnect 모듈을 통해 안전하게 접속
conn = dbconnect.MydbConnect('seoul_urban_lab')

try:
    # 2. 쿼리 작성
    # 중복을 제거(DISTINCT)하여 역코드-역명-호선명 조합만 가져옵니다.
    # stnNm은 검증용으로 같이 가져옵니다.
    sql = """
        SELECT DISTINCT stnCd, stnNm, lineNm 
        FROM subway_traffic_log
        WHERE lineNm IS NOT NULL AND lineNm != ''
        ORDER BY stnCd
    """
    
    # 3. 데이터 조회 및 DataFrame 변환
    # pandas의 read_sql을 사용하면 DB 데이터를 바로 DF로 만들 수 있습니다.
    df_mapping = pd.read_sql(sql, conn)

    # 4. 데이터 정제 (혹시 모를 중복 제거)
    # stnCd가 같은데 호선이 여러 개인 경우(환승역 등)를 대비해
    # 가장 최근에 수집된 정보를 신뢰하거나, 중복을 그대로 둡니다.
    # 여기서는 매핑의 정확도를 위해 중복을 제거하되, 첫 번째 발견된 것을 유지합니다.
    df_mapping = df_mapping.drop_duplicates(subset=['stnCd'], keep='first')
    
    # 5. CSV 파일로 저장
    file_name = 'station_master_mapping.csv'
    df_mapping.to_csv(file_name, index=False, encoding='utf-8-sig')
    
    print(f"\n파일 생성 완료: {file_name}")
    print(f"역 개수: {len(df_mapping)}")
    print("-" * 50)
    
    # 미리보기
    print(df_mapping.head(10))
    print("-" * 50)
    
except Exception as e:
    print(f"에러: {e}")
    
finally:
    conn.close()


🚀 [Master Data] 호선 매핑 테이블 생성 시작...
데이터베이서 연결 시작!
MySQL DB 연결 성공

✅ 매핑 파일 생성 완료: station_master_mapping.csv
   - 총 매핑 가능 역 개수: 276
--------------------------------------------------
  stnCd         stnNm lineNm
0  0150           서울역    1호선
1  0151            시청    1호선
2  0152            종각    1호선
3  0153          종로3가    1호선
4  0154          종로5가    1호선
5  0155           동대문    1호선
6  0156           신설동    1호선
7  0157           제기동    1호선
8  0158  청량리(서울시립대입구)    1호선
9  0159           동묘앞    1호선
--------------------------------------------------


C:\Users\Admin\AppData\Local\Temp\ipykernel_15348\1339011976.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mapping = pd.read_sql(sql, conn)
